In [ ]:
from langchain_community.llms import Ollama
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from bs4 import BeautifulSoup as Soup
from langchain.utils.html import (PREFIXES_TO_IGNORE_REGEX,SUFFIXES_TO_IGNORE_REGEX)
import os
import sys

In [2]:

# vector index persist directory
INDEX_PERSIST_DIRECTORY = os.getenv('INDEX_PERSIST_DIRECTORY', "./data/chromadb")
global conversation
conversation = None

In [3]:
# Connect to your ChromaDB and delete all exisitng docs
client = Chroma(persist_directory=INDEX_PERSIST_DIRECTORY)
# Get a handle to the default collection
collection = client.delete_collection()

/tmp/ipykernel_333064/1394225330.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  client = Chroma(persist_directory=INDEX_PERSIST_DIRECTORY)


In [4]:
#Define the documents and store it in vector database-Chroma

documents = RecursiveUrlLoader(
    url="https://www.bbc.com/",
    max_depth=1,  # Limit to top-level articles
    extractor=lambda x: Soup(x, "html.parser").find("article").text,
    # drop trailing / to avoid duplicate pages.
    link_regex=(
        f"href=[\"']{PREFIXES_TO_IGNORE_REGEX}((?:{SUFFIXES_TO_IGNORE_REGEX}.)*?)"
        r"(?:[\#'\"]|\/[\#'\"])"
    ),
).load()


# this chunk_size and chunk_overlap effects to the prompt size
# exceed promt size causes error `prompt size exceeds the context window size and cannot be processed`
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
documents = text_splitter.split_documents(documents)

# create embeddings with huggingface embedding model `all-MiniLM-L6-v2`
# then persist the vector index on vector db
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=INDEX_PERSIST_DIRECTORY
)
vectordb.persist()

/tmp/ipykernel_333064/369530036.py:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/tmp/ipykernel_333064/369530036.py:26: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [5]:
# Retrieve all documents
all_documents = vectordb.similarity_search(query="", k=len(documents))  # Empty query retrieves all documents

# Display the stored documents
for idx, doc in enumerate(all_documents):
    print(f"Document {idx+1}:")
    print(doc.page_content[:400])  # Display the first 500 characters of each document
    print("-" * 80)

Document 1:
accompanied by her grandson Beau, welcomed the arrival of the festive fir hailing from North Carolina.2 days agoUS & CanadaScience and HealthWhy we feel the need to feast togetherFor thousands of years, humans have come together in small group to feast on food. Why is it important – and why do we still continue the tradition?See moreWatch listNine of the best TV shows to watch in DecemberFrom the 
--------------------------------------------------------------------------------
Document 2:
world newsNamibians vote as ruling party seeks to extend 34-year rule34 mins agoAfricaCouncil back Creeslough explosion public inquiry call2 hrs agoEuropeThe millions of euros 'vital' to cross-border communities6 hrs agoEuropeAfrica's incoming health boss dies aged 55Faustine Ndugulile was elected WHO African director months ago and was due to take over from February.6 hrs agoAfricaVideoSee Nasa's
--------------------------------------------------------------------------------
Document 3:


In [8]:
def init_conversation():
    global conversation

    # load index
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectordb = Chroma(persist_directory=INDEX_PERSIST_DIRECTORY,embedding_function=embeddings)

    # ollama expose an api for the llam in `localhost:11434`
    llm = Ollama(
        model="llama3:8b-instruct-q6_K",
        base_url="http://localhost:11434",
        verbose=True,
    )

    # create conversation
    conversation = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=vectordb.as_retriever(),
        return_source_documents=True,
        verbose=True,
    )


def chat(question, user_id):
    global conversation

    chat_history = []
    response = conversation({"question": question, "chat_history": chat_history})
    answer = response['answer']

    #logging.info("got response from llm - %s", answer)

    # TODO save history

    return answer

In [9]:
init_conversation()

/tmp/ipykernel_333064/3186980998.py:9: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


In [13]:
chat("What is the current situation in Lebanon","user123")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

as people return to southern LebanonThe BBC's Middle East correspondent, Hugo Bachega, is at the main highway between Beirut and the south as people start to return to their homes.7 hrs agoMiddle EastWatch: Birth of seal pup caught by wildlife cameraThe camera live-streams from the heart of England’s largest grey seal colony.4 hrs agoNorfolkPeople in Israel and Lebanon react to ceasefire dealPeople in Tel Aviv and Beirut shared feelings of relief and doubt as they reacted to the news.16 hrs agoMiddle EastUS travellers pack airports ahead of Thanksgiving holidayAround 50,000 flights are expected to fly on Wednesday, according to the Federal Aviation Administration.17 hrs agoUS & CanadaBiden confirms 'good news' of Israel-

'According to the context, a ceasefire deal has been reached between Israel and Hezbollah in Lebanon, and thousands of families displaced by the conflict are traveling back to their homes. This is reported in articles such as "LIVE\'It\'s all gone\': Lebanese civilians return to destroyed homes as ceasefire appears to hold" and "BBC on the ground as people return to southern Lebanon".'